In [45]:
%config IPCompleter.greedy=True
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from nltk.translate.bleu_score import sentence_bleu
import scipy.sparse
import json

In [66]:
k_best = 20

def get_line_from_file_by_id(filename, id):
    with open(filename) as inf:
        for i, line in enumerate(inf):
            if i == id:
                return line
            
def get_line_from_filelist_by_id(filelist, id):
    return filelist[id]

with open('data/train.diff', 'r') as inf:
    train_diff_list = []
    for line in inf:
        train_diff_list.append(line)

with open('data/train.msg', 'r') as inf:
    train_msg_list = []
    for line in inf:
        train_msg_list.append(line)
            
#def extract_train_diff(id):
#    return get_line_from_file_by_id('data/train.diff', id)

def extract_train_diff(id):
    return get_line_from_filelist_by_id(train_diff_list, id)
            
#def extract_target_commit_message(id):
#    return get_line_from_file_by_id('data/train.msg', id)

def extract_target_commit_message(id):
    return get_line_from_filelist_by_id(train_msg_list, id)

In [47]:
new_diffs = []
with open('data/test.diff', 'r') as inf:
    for line in inf:
        new_diffs.append(line.strip())
#new_diffs

In [48]:
%%time
#load vocabulary
with open('data/vocabulary.json', 'r') as inf:
    vocabulary = json.load(inf) 
vectorizer = CountVectorizer(vocabulary=vocabulary, token_pattern=r'\S+', stop_words=['<nl>'])

analyzer = vectorizer.build_analyzer()

#load bag-of-words matrix
train_bow_matrix = scipy.sparse.load_npz('data/bow_matrix.npz')
#train_bow_matrix.toarray()

CPU times: user 179 ms, sys: 56.5 ms, total: 235 ms
Wall time: 240 ms


In [49]:
%%time
newdiff_bow_matrix = vectorizer.transform(new_diffs)
#newdiff_bow_matrix.shape

CPU times: user 1.53 s, sys: 19.7 ms, total: 1.55 s
Wall time: 1.56 s


In [50]:
%%time
cosine_matrix = cosine_distances(newdiff_bow_matrix, train_bow_matrix)
#cosine_matrix.shape

CPU times: user 28.2 s, sys: 9.82 s, total: 38 s
Wall time: 41.9 s


In [67]:
%%time
candidates_indices = [(row.argsort()[:k_best]).tolist() for row in cosine_matrix]
#len(candidates_indices[0]), len(candidates_indices)

CPU times: user 22.2 s, sys: 25 ms, total: 22.3 s
Wall time: 22.3 s


In [62]:
cosine_matrix.shape

(5581, 45897)

In [63]:
def choose_best(diff, ids):
    best_bleu = 0.0
    best_id = ids[0]
    for id in ids:
        reference = extract_train_diff(id)
        score = sentence_bleu([analyzer(reference)], analyzer(diff))
        if score > best_bleu:
            best_bleu = score
            best_id = id
    return best_id

In [68]:
%%time
messages_id = []
for i, diff in enumerate(new_diffs):
    cur_id = choose_best(diff, candidates_indices[i])
    messages_id.append(cur_id)
    if i % 100 == 0:
        print(i)
    
#print(new_diffs, extract_train_diff(0), extract_train_diff(1), extract_train_diff(2), extract_train_diff(3))
    
#len(messages_id)

0


/Users/ne0n/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


100
200
300
400
500
600
700
800
900
1000
1100
1200


/Users/ne0n/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700


/Users/ne0n/miniconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


4800
4900
5000
5100
5200
5300
5400
5500
CPU times: user 5min 3s, sys: 612 ms, total: 5min 4s
Wall time: 5min 5s


In [69]:
with open('data/generated.msg', 'w') as ouf:
    for i, message_id in enumerate(messages_id):
        message = extract_target_commit_message(message_id)
        ouf.write(message)
        #if i % 100 == 0:
        #    print(i)